In [13]:
import random

class Lymphocyte:
    def __init__(self, antibody):
        self.antibody = antibody
        self.search_table = self._build_table()  # Construir tabla de búsqueda para el algoritmo de detección
        self.age = 0  # Edad del linfocito (no utilizada)
        self.stimulation = 0  # Estimulación del linfocito (controla la activación)

    def _build_table(self):
        # Implementación del algoritmo KMP para construir la tabla de búsqueda
        result = [-1, 0]
        pos, cnd = 2, 0
        while pos < len(self.antibody):
            if self.antibody[pos - 1] == self.antibody[cnd]:
                cnd += 1
                result.append(cnd)
                pos += 1
            elif cnd > 0:
                cnd = result[cnd]
            else:
                result.append(0)
                pos += 1
        return result

    def detects(self, pattern):
        # Método para detectar si el linfocito reconoce un antígeno en un patrón dado
        m = 0
        i = 0
        while m + i < len(pattern):
            if self.antibody[i] == pattern[m + i]:
                if i == len(self.antibody) - 1:
                    return True
                i += 1
            else:
                m = m + i - self.search_table[i]
                if self.search_table[i] > -1:
                    i = self.search_table[i]
                else:
                    i = 0
        return False

    def __str__(self):
        # Método para representar el linfocito como una cadena de texto
        s = "antibody = "
        for bit in self.antibody:
            s += "1 " if bit else "0 "
        s += " age = " + str(self.age)
        s += "  stimulation = " + str(self.stimulation)
        return s

def load_self_set():
    # Función para cargar el conjunto de antígenos auto-reconocidos (self-set)
    result = []
    self0 = [True, False, False, True, False, True, True, False, True, False, False, True]
    # Etc.
    self5 = [False, False, True, False, True, False, True, False, False, True, False, False]
    result.extend([self0, self5])  # Agregar más si es necesario
    return [Lymphocyte(antibody) for antibody in result]

def show_self_set(self_set):
    # Función para mostrar el conjunto de antígenos auto-reconocidos
    for idx, bit_array in enumerate(self_set):
        print(idx, ": ", bit_array)

def create_lymphocyte_set(self_set, num_antibody_bits, num_lymphocytes):
    # Función para crear el conjunto de linfocitos utilizando selección negativa y detección de r-chunks
    result = []
    contents = set()
    while len(result) < num_lymphocytes:
        antibody = random_bit_array(num_antibody_bits)
        lymphocyte = Lymphocyte(antibody)
        hash_value = hash(lymphocyte)
        if not detects_any(self_set, lymphocyte) and hash_value not in contents:
            result.append(lymphocyte)
            contents.add(hash_value)
    return result

def random_bit_array(num_bits):
    # Función para generar un arreglo de bits aleatorio
    return [random.choice([True, False]) for _ in range(num_bits)]

def detects_any(self_set, lymphocyte):
    # Función para determinar si algún antígeno del conjunto self-set es detectado por un linfocito
    for pattern in self_set:
        if lymphocyte.detects(pattern.antibody):
            return True
    return False

def remove_detected_antigens(self_set, lymphocyte_set):
    # Función para eliminar los antígenos detectados por los linfocitos del conjunto self-set
    removed_antigens = []
    intruders_detected = 0
    for lymphocyte in lymphocyte_set:
        for pattern in self_set:
            if lymphocyte.detects(pattern.antibody):
                removed_antigens.append(pattern)
                self_set.remove(pattern)
                intruders_detected += 1
                break
    return removed_antigens, intruders_detected

def show_lymphocyte_set(lymphocyte_set):
    # Función para mostrar el conjunto de linfocitos
    for idx, lymphocyte in enumerate(lymphocyte_set):
        print(idx, ": ", lymphocyte)

def main():
    print("\nBegin Artificial Immune System for Intrusion Detection demo\n")
    random.seed(1)

    num_pattern_bits = 12
    num_antibody_bits = 4
    num_lymphocytes = 3
    stimulation_threshold = 3

    print("Loading self-antigen set ('normal' historical patterns)")
    self_set = load_self_set()
    show_self_set(self_set)

    print("\nCreating lymphocyte set using negative selection and r-chunks detection")
    lymphocyte_set = create_lymphocyte_set(self_set, num_antibody_bits, num_lymphocytes)
    show_lymphocyte_set(lymphocyte_set)

    print("\nBegin AIS intrusion detection simulation\n")
    time = 0
    max_time = 6
    while time < max_time:
        print("============================================")
        incoming = random_bit_array(num_pattern_bits)
        print("Incoming pattern = ", incoming, "\n")
        for idx, lymphocyte in enumerate(lymphocyte_set):
            if lymphocyte.detects(incoming):
                print("Incoming pattern detected by lymphocyte ", idx)
                lymphocyte.stimulation += 1
                if lymphocyte.stimulation >= stimulation_threshold:
                    print("Lymphocyte ", idx, " stimulated! Check incoming as possible intrusion!")
                else:
                    print("Lymphocyte ", idx, " not over stimulation threshold")
        time += 1
        print("============================================")

    print("\nRemoving detected antigens...")
    removed_antigens, intruders_detected = remove_detected_antigens(self_set, lymphocyte_set)
    print("Removed antigens:")
    for antigen in removed_antigens:
        print(antigen)
    print("Number of intruders detected:", intruders_detected)

    print("\nEnd AIS IDS demo\n")

if __name__ == "__main__":
    main()



Begin Artificial Immune System for Intrusion Detection demo

Loading self-antigen set ('normal' historical patterns)
0 :  antibody = 1 0 0 1 0 1 1 0 1 0 0 1  age = 0  stimulation = 0
1 :  antibody = 0 0 1 0 1 0 1 0 0 1 0 0  age = 0  stimulation = 0

Creating lymphocyte set using negative selection and r-chunks detection
0 :  antibody = 0 0 0 0  age = 0  stimulation = 0
1 :  antibody = 1 1 1 1  age = 0  stimulation = 0
2 :  antibody = 1 1 0 0  age = 0  stimulation = 0

Begin AIS intrusion detection simulation

Incoming pattern =  [False, True, False, True, False, False, True, True, True, True, True, True] 

Incoming pattern detected by lymphocyte  1
Lymphocyte  1  not over stimulation threshold
Incoming pattern =  [False, False, False, False, False, True, False, True, True, False, True, False] 

Incoming pattern detected by lymphocyte  0
Lymphocyte  0  not over stimulation threshold
Incoming pattern =  [False, True, False, False, False, False, False, True, False, False, True, True] 

I